# Block Model using `emg3d`

### Note regarding runtime

The following environment variables were set before starting Jupyter:
```
export OMP_NUM_THREADS=1
export MKL_NUM_THREADS=1
export OPENBLAS_NUM_THREADS=1
```
This ensures that our code runs only on one thread. CPU-time is therefore the same as walltime (or even a tiny fraction smaller).

In [1]:
import emg3d
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.pyplot as plt
%load_ext memory_profiler

In [2]:
%matplotlib notebook
plt.style.use('ggplot')

## Load model and survey

In [3]:
ds = xr.load_dataset('../block_model_and_survey.nc', engine='h5netcdf')

### Extract required info

In [4]:
# Mesh
hx, hy, hz = ds.attrs['hx'], ds.attrs['hy'], ds.attrs['hz']
x0 = ds.attrs['x0']
mesh_model = emg3d.utils.TensorMesh([hx, hy, hz], x0=x0)

# Block Model
resh, resv = ds.attrs['resh_tg'], ds.attrs['resv_tg']

# Survey
src = ds.attrs['src']
strength = ds.attrs['strength']
freq = ds.attrs['freq']
rec_x = ds.x.data
rec_y = ds.attrs['rec_y']
rec_z = ds.attrs['rec_z']

# Calculate source center (for meshing)
src_c = np.mean(src.reshape(-1, 2), 1)

## `emg3d` computation

For the background model, we could get away with a much coarser mesh. However, for simplicity we just use the same mesh for the background model and the full model.

Also, for the 1D model we only calculate the receiver lines for y=0 and y=-3 km, as the result for y=+/-3 km are identical.

In [5]:
# Get cell widths and origin in each direction
xx, x0 = emg3d.utils.get_hx_h0(
    freq=freq, res=[0.3, 1000], fixed=src_c[0], domain=[-4500, 4500], min_width=100)
yy, y0 = emg3d.utils.get_hx_h0(
    freq=freq, res=[0.3, 1000], fixed=src_c[1], domain=[-4000, 4000], min_width=100)
zz, z0 = emg3d.utils.get_hx_h0(
    freq=freq, res=[0.3, 100, 1000], fixed=rec_z, domain=[-3200, 0], min_width=100)

   Skin depth (m/l-r)  [m] : 276 / 15916
   Survey domain       [m] : -4500 - 4500
   Calculation domain  [m] : -100000 - 100000
   Final extent        [m] : -106581 - 106581
   Min/max cell width  [m] : 100 / 100 / 26001
   Alpha survey/calc       : 1.000 / 1.340
   Number of cells (s/c/r) : 128 (90/38/0)

   Skin depth (m/l-r)  [m] : 276 / 15916
   Survey domain       [m] : -4000 - 4000
   Calculation domain  [m] : -100000 - 100000
   Final extent        [m] : -109379 - 109379
   Min/max cell width  [m] : 100 / 100 / 21176
   Alpha survey/calc       : 1.000 / 1.250
   Number of cells (s/c/r) : 128 (80/48/0)

   Skin depth (m/l/r)  [m] : 276 / 5033 / 15916
   Survey domain       [m] : -3200 - 0
   Calculation domain  [m] : -33523 - 99400
   Final extent        [m] : -35828 - 119289
   Min/max cell width  [m] : 100 / 100 / 32948
   Alpha survey/calc       : 1.000 / 1.380
   Number of cells (s/c/r) : 64 (32/32/0)



In [12]:
# Create a TensorMesh instance.
mesh = emg3d.utils.TensorMesh([xx, yy, zz], x0=[x0, y0, z0])

In [7]:
# Interpolate to new mesh
cresh = emg3d.utils.grid2grid(mesh_model, resh, mesh, 'volume', log=True)
cresv = emg3d.utils.grid2grid(mesh_model, resv, mesh, 'volume', log=True)

# Create model
model = emg3d.utils.Model(mesh, res_x=cresh, res_z=cresv)

### 3. Full model calculation

In [8]:
# Source field
sfield = emg3d.utils.get_source_field(mesh, src, freq, strength=strength)

solver_inp = {'sslsolver': True, 'semicoarsening': True,
              'linerelaxation': True, 'return_info': True}

In [9]:
mem = %memit -o efield, info = emg3d.solve(mesh, model, sfield, verb=3, **solver_inp)
time = f"{info['time']:.0f} s"
ram = f"{mem.mem_usage[0] - mem.baseline:.0f} MiB"
print(f"memory usage: {ram}")


:: emg3d START :: 15:24:18 :: v0.9.3

   MG-cycle       : 'F'                 sslsolver : 'bicgstab'
   semicoarsening : True [1 2 3]        tol       : 1e-06
   linerelaxation : True [4 5 6]        maxit     : 50 (3)
   nu_{i,1,c,2}   : 0, 2, 1, 2          verb      : 3
   Original grid  : 128 x 128 x  64     => 1,048,576 cells
   Coarsest grid  :   2 x   2 x   2     => 8 cells
   Coarsest level :   6 ;   6 ;   5   

   [hh:mm:ss]  rel. error            solver              MG          l s

       h_
      2h_ \                                        /
      4h_  \                            /\        / 
      8h_   \                  /\      /  \      /  
     16h_    \          /\    /  \    /    \    /   
     32h_     \    /\  /  \  /    \  /      \  /    
     64h_      \/\/  \/    \/      \/        \/     

   [15:24:35]   2.722e-03  after                       1 F-cycles    4 1
   [15:24:52]   2.110e-04  after                       2 F-cycles    5 2
   [15:25:08]   4.033e-05  a

In [10]:
# Extract (interpolate) Ex-field at receiver locations from the emg3d result.
egd = np.zeros((rec_x.size, 3), dtype=complex)
for i, y in enumerate(rec_y):
    egd[:, i] = emg3d.utils.get_receiver(mesh, efield.fx, (rec_x, y, rec_z))

## Save data

In [14]:
# Save the three lines
ds.line_1_re.data = egd[:, 0].real
ds.line_1_im.data = egd[:, 0].imag
ds.line_2_re.data = egd[:, 1].real
ds.line_2_im.data = egd[:, 1].imag
ds.line_3_re.data = egd[:, 2].real
ds.line_3_im.data = egd[:, 2].imag

# Add info
ds.attrs['runtime'] = time
ds.attrs['cputime'] = time  # Run on 1 thread, so cputime <~ runtime
ds.attrs['nthreads'] = 1
ds.attrs['maxram'] = ram
ds.attrs['ncells'] = f"({mesh.nCx} x {mesh.nCy} x {mesh.nCz}) - {mesh.nC}"
ds.attrs['nnodes'] = 'N/A'
ds.attrs['ndof'] = 'N/A'
ds.attrs['extent'] = (f"x = {mesh.vectorNx[0]:.1f}-{mesh.vectorNx[-1]:.1f}; " # Total mesh extent
                      f"y = {mesh.vectorNy[0]:.1f}-{mesh.vectorNy[-1]:.1f}; "
                      f"z = {mesh.vectorNz[0]:.1f}-{mesh.vectorNz[-1]:.1f}")
ds.attrs['min_cwidth'] = f"({mesh.hx.min():.1f} x {mesh.hy.min():.1f} x {mesh.hz.min():.1f}"
ds.attrs['max_cwidth'] = f"({mesh.hx.max():.1f} x {mesh.hy.max():.1f} x {mesh.hz.max():.1f}"
ds.attrs['nedges'] = 'N/A'
ds.attrs['machine'] = "Laptop; i7-6600U CPU@2.6 GHz x4; 16 GB of memory, Ubuntu 18.04"
ds.attrs['version'] = emg3d.__version__
ds.attrs['date'] = datetime.today().isoformat()

# Save it under <{model}_{code}.nc>
ds.to_netcdf(f"../results/block_emg3d.nc", engine='h5netcdf')

In [ ]:
emg3d.Report(xr)